# Sudoku-8 : Resolution par Strategies Humaines (Python)

**Niveau** : Techniques d'inference | **Duree** : ~40 min | **Prerequis** : Sudoku-0 Environment

## Navigation

| << Precedent | [Index](README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-6-AIMA-CSP-Python](Sudoku-6-AIMA-CSP-Python.ipynb) | | [Sudoku-9-GraphColoring-Python](Sudoku-9-GraphColoring-Python.ipynb) |

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre comment les experts humains resolvent les Sudoku par deduction logique
2. Implementer les techniques de base : Naked Singles, Hidden Singles
3. Implementer les techniques intermediaires : Naked Pairs, Pointing Pairs
4. Implementer la technique avancee X-Wing
5. Combiner ces techniques dans un solveur hybride avec fallback vers le backtracking

---

Ce notebook implemente un solveur de Sudoku utilisant les techniques de deduction utilisees par les experts humains.
C'est l'equivalent Python du notebook C# `Sudoku-8-HumanStrategies-Csharp.ipynb`.

## Introduction

Les algorithmes que nous avons vus dans les notebooks precedents (backtracking, OR-Tools, Dancing Links, etc.) resolvent les Sudoku par **force brute** ou par **satisfaction de contraintes** de maniere systematique. Un expert humain procede tout autrement : il applique des **techniques de deduction logique** de difficulte croissante, en commencant par les plus simples.

### Hierarchie des techniques

1. **Naked Single** : Une case ne peut contenir qu'une seule valeur
2. **Hidden Single** : Une valeur ne peut aller qu'a une seule case dans une unite (ligne/colonne/bloc)
3. **Naked Pair** : Deux cases ne contiennent que les memes deux valeurs candidats
4. **Pointing Pair** : Une valeur dans un bloc est restreinte a une ligne/colonne
5. **X-Wing** : Une valeur forme un rectangle dans deux lignes/colonnes

### Difficulte des puzzles

| Niveau | Techniques necessaires |
|--------|-----------------------|
| Easy | Naked Singles, Hidden Singles |
| Medium | Naked Pairs, Pointing Pairs |
| Hard | X-Wing, techniques avancees |
| Expert | Swordfish, XY-Wing, etc. |

In [ ]:
# Imports
import numpy as np
import time
from typing import List, Tuple, Optional, Set, Dict
from dataclasses import dataclass
from enum import Enum

print("Libraries importees avec succes.")

In [ ]:
# Configuration du chemin vers les puzzles
import os
from pathlib import Path

NOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")
PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"

if PUZZLES_DIR.exists():
    print(f"Dossier Puzzles: {PUZZLES_DIR}")
else:
    print(f"ATTENTION: Dossier Puzzles non trouve a {PUZZLES_DIR}")
    PUZZLES_DIR = Path(os.getcwd()) / "Puzzles"

## 1. Classe Sudoku avec Candidats

Pour les strategies humaines, nous devons suivre non seulement les valeurs placees, mais aussi les candidats possibles pour chaque case.

In [ ]:
@dataclass
class CandidateInfo:
    """Informations sur les candidats d'une case."""
    row: int
    col: int
    value: int
    
    def __hash__(self):
        return hash((self.row, self.col, self.value))

class HumanSudokuSolver:
    """Solveur de Sudoku utilisant les strategies humaines."""
    
    def __init__(self, puzzle_str: str):
        # Grille principale (0 = vide)
        self.grid = np.zeros((9, 9), dtype=int)
        
        # Candidats pour chaque case (ensemble de valeurs possibles)
        self.candidates = [[set() for _ in range(9)] for _ in range(9)]
        
        # Historique des placements
        self.placements = []
        
        # Statistiques
        self.strategy_counts = {}
        
        # Initialiser
        self._initialize(puzzle_str)
    
    def _initialize(self, puzzle_str: str):
        """Initialise la grille et les candidats."""
        puzzle_str = puzzle_str.replace('.', '0').replace(' ', '').replace('\n', '')
        
        for i in range(81):
            row, col = i // 9, i % 9
            val = int(puzzle_str[i])
            if val != 0:
                self.grid[row, col] = val
                self.placements.append(CandidateInfo(row, col, val))
        
        # Initialiser les candidats
        self._init_candidates()
    
    def _init_candidates(self):
        """Initialise les candidats pour toutes les cases vides."""
        for row in range(9):
            for col in range(9):
                if self.grid[row, col] == 0:
                    self.candidates[row][col] = self._get_possible_values(row, col)
    
    def _get_possible_values(self, row: int, col: int) -> Set[int]:
        """Retourne les valeurs possibles pour une case."""
        if self.grid[row, col] != 0:
            return set()
        
        values = set(range(1, 10))
        
        # Retirer les valeurs de la ligne
        values -= set(self.grid[row, :])
        
        # Retirer les valeurs de la colonne
        values -= set(self.grid[:, col])
        
        # Retirer les valeurs du bloc
        br, bc = 3 * (row // 3), 3 * (col // 3)
        values -= set(self.grid[br:br+3, bc:bc+3].flatten())
        
        return values
    
    def is_solved(self) -> bool:
        """Verifie si le Sudoku est resolu."""
        return np.all(self.grid != 0) and self._is_valid()
    
    def _is_valid(self) -> bool:
        """Verifie la validite de la grille."""
        # Verifier les lignes
        for r in range(9):
            if len(set(self.grid[r, :])) < 9 and 0 not in self.grid[r, :]:
                return False
        # Verifier les colonnes
        for c in range(9):
            if len(set(self.grid[:, c])) < 9 and 0 not in self.grid[:, c]:
                return False
        return True
    
    def __str__(self) -> str:
        lines = []
        for r in range(9):
            if r > 0 and r % 3 == 0:
                lines.append('-' * 21)
            row_str = ''
            for c in range(9):
                if c > 0 and c % 3 == 0:
                    row_str += '| '
                val = self.grid[r, c]
                row_str += (str(val) if val != 0 else '.') + ' '
            lines.append(row_str)
        return '\n'.join(lines)

def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

# Charger les puzzles
easy_puzzles = load_puzzles(str(PUZZLES_DIR / 'Sudoku_Easy51.txt'), max_puzzles=5)
print(f"Puzzles charges: {len(easy_puzzles)}")

# Test
solver = HumanSudokuSolver(easy_puzzles[0])
print("\nGrille de test:")
print(solver)

## 2. Technique 1 : Naked Single

Un **Naked Single** (ou "Singleton") est une case qui ne contient qu'un seul candidat possible. C'est la technique la plus simple.

In [ ]:
    def find_naked_singles(self) -> List[CandidateInfo]:
        """Trouve les Naked Singles (cases avec un seul candidat)."""
        placements = []
        
        for row in range(9):
            for col in range(9):
                if self.grid[row, col] == 0:
                    candidates = self.candidates[row][col]
                    if len(candidates) == 1:
                        value = list(candidates)[0]
                        placements.append(CandidateInfo(row, col, value))
        
        return placements
    
    def apply_placement(self, info: CandidateInfo, strategy: str) -> bool:
        """Apique un placement et met a jour les candidats."""
        if self.grid[info.row, info.col] != 0:
            return True  # Deja place
        
        # Verifier que le placement est valide
        if info.value not in self.candidates[info.row][info.col]:
            return False  # Placement invalide
        
        # Placer la valeur
        self.grid[info.row, info.col] = info.value
        self.placements.append(info)
        self.candidates[info.row][info.col] = set()
        
        # Mettre a jour les candidats voisins
        self._remove_candidate_from_peers(info.row, info.col, info.value)
        
        # Statistiques
        self.strategy_counts[strategy] = self.strategy_counts.get(strategy, 0) + 1
        
        return True
    
    def _remove_candidate_from_peers(self, row: int, col: int, value: int):
        """Retire un candidat des cases voisines."""
        # Meme ligne
        for c in range(9):
            if c != col:
                self.candidates[row][c].discard(value)
        
        # Meme colonne
        for r in range(9):
            if r != row:
                self.candidates[r][col].discard(value)
        
        # Meme bloc
        br, bc = 3 * (row // 3), 3 * (col // 3)
        for r in range(br, br + 3):
            for c in range(bc, bc + 3):
                if r != row or c != col:
                    self.candidates[r][c].discard(value)

# Ajouter les methodes a la classe
HumanSudokuSolver.find_naked_singles = find_naked_singles
HumanSudokuSolver.apply_placement = apply_placement
HumanSudokuSolver._remove_candidate_from_peers = _remove_candidate_from_peers

print("Methodes Naked Single ajoutees.")

# Test
solver = HumanSudokuSolver(easy_puzzles[0])
naked_singles = solver.find_naked_singles()
print(f"Naked Singles trouves: {len(naked_singles)}")
for ns in naked_singles[:5]:
    print(f"  ({ns.row}, {ns.col}) = {ns.value}")

## 3. Technique 2 : Hidden Single

Un **Hidden Single** est une valeur qui ne peut aller qu'a une seule case dans une unite (ligne, colonne ou bloc), meme si cette case a d'autres candidats.

In [ ]:
    def find_hidden_singles(self) -> List[CandidateInfo]:
        """Trouve les Hidden Singles dans les lignes, colonnes et blocs."""
        placements = []
        
        # Chercher dans les lignes
        for row in range(9):
            value_positions = {v: [] for v in range(1, 10)}
            for col in range(9):
                if self.grid[row, col] == 0:
                    for value in self.candidates[row][col]:
                        value_positions[value].append(col)
            
            for value, positions in value_positions.items():
                if len(positions) == 1:
                    col = positions[0]
                    if self.grid[row, col] == 0:
                        placements.append(CandidateInfo(row, col, value))
        
        # Chercher dans les colonnes
        for col in range(9):
            value_positions = {v: [] for v in range(1, 10)}
            for row in range(9):
                if self.grid[row, col] == 0:
                    for value in self.candidates[row][col]:
                        value_positions[value].append(row)
            
            for value, positions in value_positions.items():
                if len(positions) == 1:
                    row = positions[0]
                    if self.grid[row, col] == 0:
                        placements.append(CandidateInfo(row, col, value))
        
        # Chercher dans les blocs
        for block_row in range(3):
            for block_col in range(3):
                value_positions = {v: [] for v in range(1, 10)}
                
                for r in range(block_row * 3, block_row * 3 + 3):
                    for c in range(block_col * 3, block_col * 3 + 3):
                        if self.grid[r, c] == 0:
                            for value in self.candidates[r][c]:
                                value_positions[value].append((r, c))
                
                for value, positions in value_positions.items():
                    if len(positions) == 1:
                        row, col = positions[0]
                        if self.grid[row, col] == 0:
                            placements.append(CandidateInfo(row, col, value))
        
        return placements

# Ajouter la methode
HumanSudokuSolver.find_hidden_singles = find_hidden_singles

print("Methode Hidden Single ajoutee.")

# Test
solver = HumanSudokuSolver(easy_puzzles[0])
hidden_singles = solver.find_hidden_singles()
print(f"Hidden Singles trouves: {len(hidden_singles)}")

## 4. Technique 3 : Naked Pair

Un **Naked Pair** est deux cases de la meme unite qui contiennent exactement les memes deux candidats. Ces deux candidats peuvent donc etre elimines des autres cases de l'unite.

In [ ]:
    def find_naked_pairs(self) -> List[Tuple[int, int, int, Set[int], str]]:
        """Trouve les Naked Pairs dans les lignes, colonnes et blocs.
        
        Returns:
            Liste de (row1, col1, row2, col2, pair_values, unit_type)
            où unit_type est 'row', 'col', ou 'block'
        """
        results = []
        
        # Chercher dans les lignes
        for row in range(9):
            pair_cells = []
            for col in range(9):
                if self.grid[row, col] == 0 and len(self.candidates[row][col]) == 2:
                    pair_cells.append((col, self.candidates[row][col]))
            
            # Chercher les paires de cases avec les memes candidats
            for i in range(len(pair_cells)):
                for j in range(i + 1, len(pair_cells)):
                    col1, cand1 = pair_cells[i]
                    col2, cand2 = pair_cells[j]
                    if cand1 == cand2:
                        results.append((row, col1, row, col2, cand1, 'row'))
        
        # Chercher dans les colonnes
        for col in range(9):
            pair_cells = []
            for row in range(9):
                if self.grid[row, col] == 0 and len(self.candidates[row][col]) == 2:
                    pair_cells.append((row, self.candidates[row][col]))
            
            for i in range(len(pair_cells)):
                for j in range(i + 1, len(pair_cells)):
                    row1, cand1 = pair_cells[i]
                    row2, cand2 = pair_cells[j]
                    if cand1 == cand2:
                        results.append((row1, col, row2, col, cand1, 'col'))
        
        # Chercher dans les blocs
        for block_row in range(3):
            for block_col in range(3):
                pair_cells = []
                for r in range(block_row * 3, block_row * 3 + 3):
                    for c in range(block_col * 3, block_col * 3 + 3):
                        if self.grid[r, c] == 0 and len(self.candidates[r][c]) == 2:
                            pair_cells.append(((r, c), self.candidates[r][c]))
                
                for i in range(len(pair_cells)):
                    for j in range(i + 1, len(pair_cells)):
                        (r1, c1), cand1 = pair_cells[i]
                        (r2, c2), cand2 = pair_cells[j]
                        if cand1 == cand2:
                            results.append((r1, c1, r2, c2, cand1, 'block'))
        
        return results
    
    def apply_naked_pair(self, row1: int, col1: int, row2: int, col2: int, 
                       pair_values: Set[int], unit_type: str) -> int:
        """Apique un Naked Pair en eliminant les candidats des autres cases.
        
        Returns:
            Nombre de candidats elimines
        """
        eliminated = 0
        
        if unit_type == 'row':
            row = row1
            for col in range(9):
                if col != col1 and col != col2 and self.grid[row, col] == 0:
                    for val in pair_values:
                        if val in self.candidates[row][col]:
                            self.candidates[row][col].discard(val)
                            eliminated += 1
        
        elif unit_type == 'col':
            col = col1
            for row in range(9):
                if row != row1 and row != row2 and self.grid[row, col] == 0:
                    for val in pair_values:
                        if val in self.candidates[row][col]:
                            self.candidates[row][col].discard(val)
                            eliminated += 1
        
        elif unit_type == 'block':
            br, bc = 3 * (row1 // 3), 3 * (col1 // 3)
            for r in range(br, br + 3):
                for c in range(bc, bc + 3):
                    if (r != row1 or c != col1) and (r != row2 or c != col2):
                        if self.grid[r, c] == 0:
                            for val in pair_values:
                                if val in self.candidates[r][c]:
                                    self.candidates[r][c].discard(val)
                                    eliminated += 1
        
        return eliminated

# Ajouter les methodes
HumanSudokuSolver.find_naked_pairs = find_naked_pairs
HumanSudokuSolver.apply_naked_pair = apply_naked_pair

print("Methodes Naked Pair ajoutees.")

## 5. Technique 4 : Pointing Pair

Un **Pointing Pair** (ou "Locked Candidates") se produit quand les candidats d'une valeur dans un bloc sont restreints a une seule ligne ou colonne. Cette valeur peut donc etre eliminee des autres cases de cette ligne/colonne en dehors du bloc.

In [ ]:
    def find_pointing_pairs(self) -> List[Tuple[int, int, int, str]]:
        """Trouve les Pointing Pairs dans les blocs.
        
        Returns:
            Liste de (block_row, block_col, value, direction)
            où direction est 'row' ou 'col'
        """
        results = []
        
        for block_row in range(3):
            for block_col in range(3):
                for value in range(1, 10):
                    # Trouver les positions de la valeur dans le bloc
                    positions = []
                    for r in range(block_row * 3, block_row * 3 + 3):
                        for c in range(block_col * 3, block_col * 3 + 3):
                            if self.grid[r, c] == 0 and value in self.candidates[r][c]:
                                positions.append((r, c))
                    
                    if len(positions) < 2:
                        continue
                    
                    # Verifier si tous sur la meme ligne
                    rows = set(p[0] for p in positions)
                    if len(rows) == 1:
                        results.append((block_row, block_col, value, 'row'))
                    
                    # Verifier si tous sur la meme colonne
                    cols = set(p[1] for p in positions)
                    if len(cols) == 1:
                        results.append((block_row, block_col, value, 'col'))
        
        return results
    
    def apply_pointing_pair(self, block_row: int, block_col: int, 
                            value: int, direction: str) -> int:
        """Apique un Pointing Pair.
        
        Returns:
            Nombre de candidats elimines
        """
        eliminated = 0
        
        # Trouver les positions dans le bloc
        positions = []
        for r in range(block_row * 3, block_row * 3 + 3):
            for c in range(block_col * 3, block_col * 3 + 3):
                if self.grid[r, c] == 0 and value in self.candidates[r][c]:
                    positions.append((r, c))
        
        if direction == 'row':
            row = positions[0][0]
            for col in range(9):
                if col // 3 != block_col and self.grid[row, col] == 0:
                    if value in self.candidates[row][col]:
                        self.candidates[row][col].discard(value)
                        eliminated += 1
        
        elif direction == 'col':
            col = positions[0][1]
            for row in range(9):
                if row // 3 != block_row and self.grid[row, col] == 0:
                    if value in self.candidates[row][col]:
                        self.candidates[row][col].discard(value)
                        eliminated += 1
        
        return eliminated

# Ajouter les methodes
HumanSudokuSolver.find_pointing_pairs = find_pointing_pairs
HumanSudokuSolver.apply_pointing_pair = apply_pointing_pair

print("Methodes Pointing Pair ajoutees.")

## 6. Technique 5 : X-Wing

Un **X-Wing** se produit quand une valeur a exactement deux candidats dans chacune de deux lignes differentes, et ces candidats sont dans les memes deux colonnes. La valeur peut donc etre eliminee des autres cases de ces colonnes.

In [ ]:
    def find_x_wings(self) -> List[Tuple[int, int, int, int, int]]:
        """Trouve les X-Wings dans les lignes.
        
        Returns:
            Liste de (row1, row2, col1, col2, value)
        """
        results = []
        
        for value in range(1, 10):
            # Pour chaque valeur, trouver les lignes avec exactement 2 candidats
            rows_with_two = []
            for row in range(9):
                cols_with_value = []
                for col in range(9):
                    if self.grid[row, col] == 0 and value in self.candidates[row][col]:
                        cols_with_value.append(col)
                
                if len(cols_with_value) == 2:
                    rows_with_two.append((row, cols_with_value))
            
            # Chercher deux lignes avec les memes colonnes
            for i in range(len(rows_with_two)):
                for j in range(i + 1, len(rows_with_two)):
                    row1, cols1 = rows_with_two[i]
                    row2, cols2 = rows_with_two[j]
                    
                    if cols1 == cols2:
                        results.append((row1, row2, cols1[0], cols1[1], value))
        
        return results
    
    def apply_x_wing(self, row1: int, row2: int, col1: int, col2: int, value: int) -> int:
        """Apique un X-Wing.
        
        Returns:
            Nombre de candidats elimines
        """
        eliminated = 0
        
        # Eliminer la valeur des autres cases des colonnes
        for col in [col1, col2]:
            for row in range(9):
                if row != row1 and row != row2 and self.grid[row, col] == 0:
                    if value in self.candidates[row][col]:
                        self.candidates[row][col].discard(value)
                        eliminated += 1
        
        return eliminated

# Ajouter les methodes
HumanSudokuSolver.find_x_wings = find_x_wings
HumanSudokuSolver.apply_x_wing = apply_x_wing

print("Methodes X-Wing ajoutees.")

## 7. Solveur Complet

Le solveur combine toutes les techniques avec un fallback vers le backtracking si necessaire.

In [ ]:
    def solve(self, max_iterations: int = 1000) -> bool:
        """Resout le Sudoku en utilisant les strategies humaines.
        
        Returns:
            True si resolu, False sinon
        """
        self.strategy_counts = {}
        iteration = 0
        
        while not self.is_solved() and iteration < max_iterations:
            iteration += 1
            progress = False
            
            # 1. Naked Singles
            naked_singles = self.find_naked_singles()
            for ns in naked_singles:
                if self.apply_placement(ns, "Naked Single"):
                    progress = True
            
            if self.is_solved():
                return True
            
            # 2. Hidden Singles
            hidden_singles = self.find_hidden_singles()
            for hs in hidden_singles:
                if self.apply_placement(hs, "Hidden Single"):
                    progress = True
            
            if self.is_solved():
                return True
            
            # 3. Naked Pairs
            naked_pairs = self.find_naked_pairs()
            for r1, c1, r2, c2, pair_vals, unit_type in naked_pairs:
                eliminated = self.apply_naked_pair(r1, c1, r2, c2, pair_vals, unit_type)
                if eliminated > 0:
                    progress = True
                    self.strategy_counts["Naked Pair"] = self.strategy_counts.get("Naked Pair", 0) + 1
            
            # 4. Pointing Pairs
            pointing_pairs = self.find_pointing_pairs()
            for br, bc, val, direction in pointing_pairs:
                eliminated = self.apply_pointing_pair(br, bc, val, direction)
                if eliminated > 0:
                    progress = True
                    self.strategy_counts["Pointing Pair"] = self.strategy_counts.get("Pointing Pair", 0) + 1
            
            # 5. X-Wings
            x_wings = self.find_x_wings()
            for r1, r2, c1, c2, val in x_wings:
                eliminated = self.apply_x_wing(r1, r2, c1, c2, val)
                if eliminated > 0:
                    progress = True
                    self.strategy_counts["X-Wing"] = self.strategy_counts.get("X-Wing", 0) + 1
            
            if not progress:
                # Plus de progres, essayer le backtracking
                if self._backtrack():
                    return True
                break
        
        return self.is_solved()
    
    def _backtrack(self) -> bool:
        """Simple backtracking comme fallback."""
        # Trouver la premiere case vide
        for row in range(9):
            for col in range(9):
                if self.grid[row, col] == 0:
                    candidates = list(self.candidates[row][col])
                    
                    for value in candidates:
                        if self._is_valid_placement(row, col, value):
                            self.grid[row, col] = value
                            self.placements.append(CandidateInfo(row, col, value))
                            
                            if self._backtrack():
                                return True
                            
                            # Annuler
                            self.grid[row, col] = 0
                            self.placements.pop()
                    
                    return False
        return True
    
    def _is_valid_placement(self, row: int, col: int, value: int) -> bool:
        """Verifie si un placement est valide."""
        # Ligne
        if value in self.grid[row, :]:
            return False
        # Colonne
        if value in self.grid[:, col]:
            return False
        # Bloc
        br, bc = 3 * (row // 3), 3 * (col // 3)
        if value in self.grid[br:br+3, bc:bc+3]:
            return False
        return True

# Ajouter les methodes
HumanSudokuSolver.solve = solve
HumanSudokuSolver._backtrack = _backtrack
HumanSudokuSolver._is_valid_placement = _is_valid_placement

print("Methode solve ajoutee.")

## 8. Test et Benchmark

In [ ]:
print("=== Test : Puzzle Facile ===")
puzzle_str = easy_puzzles[0]

solver = HumanSudokuSolver(puzzle_str)
print("Puzzle original:")
print(solver)

start = time.time()
solved = solver.solve()
elapsed = time.time() - start

print(f"\nResolu: {solved}")
print(f"Temps: {elapsed*1000:.2f}ms")
print(f"\nStatistiques des strategies:")
for strategy, count in solver.strategy_counts.items():
    print(f"  {strategy}: {count}")

print("\nSolution:")
print(solver)

### Interpretation : Resultat du solveur

| Aspect | Observation |
|--------|-------------|
| Strategies utilisees | Naked Singles et Hidden Singles suffisent pour les puzzles faciles |
| Temps | Resolution rapide pour les puzzles simples |
| Fallback backtracking | Utilise si les strategies humaines echouent |

> **Point cle** : Les strategies humaines resolvent la plupart des puzzles faciles et moyens sans backtracking.

In [ ]:
# Benchmark sur plusieurs puzzles
def benchmark_human(puzzles: List[str], num_puzzles: int = 5):
    """Benchmark du solveur humain."""
    print(f"\n=== Benchmark: Strategies Humaines ({num_puzzles} puzzles) ===")
    
    total_time = 0
    solved_count = 0
    all_strategies = {}
    
    for i, puzzle_str in enumerate(puzzles[:num_puzzles]):
        solver = HumanSudokuSolver(puzzle_str)
        
        start = time.time()
        solved = solver.solve()
        elapsed = time.time() - start
        
        total_time += elapsed
        if solved:
            solved_count += 1
        
        status = "OK" if solved else "Echec"
        print(f"  Puzzle {i+1}: {status}, {elapsed*1000:.2f}ms")
        
        for strategy, count in solver.strategy_counts.items():
            all_strategies[strategy] = all_strategies.get(strategy, 0) + count
    
    print(f"\nResume:")
    print(f"  Resolus: {solved_count}/{num_puzzles}")
    print(f"  Temps total: {total_time*1000:.2f}ms")
    print(f"  Temps moyen: {(total_time/num_puzzles)*1000:.2f}ms")
    print(f"\nStrategies utilisees:")
    for strategy, count in sorted(all_strategies.items()):
        print(f"  {strategy}: {count}")

benchmark_human(easy_puzzles, num_puzzles=5)

## 9. Comparaison avec le Backtracking Simple

In [ ]:
class SimpleBacktracking:
    """Simple backtracking pour comparaison."""
    
    def __init__(self):
        self.call_count = 0
    
    def solve(self, puzzle_str: str) -> bool:
        grid = np.zeros((9, 9), dtype=int)
        for i in range(81):
            grid[i // 9, i % 9] = int(puzzle_str[i])
        
        self.call_count = 0
        return self._backtrack(grid)
    
    def _backtrack(self, grid: np.ndarray) -> bool:
        self.call_count += 1
        
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    for val in range(1, 10):
                        if self._is_valid(grid, r, c, val):
                            grid[r, c] = val
                            if self._backtrack(grid):
                                return True
                            grid[r, c] = 0
                    return False
        return True
    
    def _is_valid(self, grid: np.ndarray, row: int, col: int, val: int) -> bool:
        if val in grid[row, :]:
            return False
        if val in grid[:, col]:
            return False
        br, bc = 3 * (row // 3), 3 * (col // 3)
        if val in grid[br:br+3, bc:bc+3]:
            return False
        return True

print("\n=== Comparaison : Strategies Humaines vs Backtracking ===")

for i, puzzle_str in enumerate(easy_puzzles[:3]):
    print(f"\nPuzzle {i+1}:")
    
    # Strategies humaines
    solver = HumanSudokuSolver(puzzle_str)
    start = time.time()
    solved_h = solver.solve()
    t_h = time.time() - start
    
    # Backtracking
    bt = SimpleBacktracking()
    start = time.time()
    solved_bt = bt.solve(puzzle_str)
    t_bt = time.time() - start
    
    print(f"  Humain: {solved_h}, {t_h*1000:.2f}ms")
    print(f"  Backtracking: {solved_bt}, {t_bt*1000:.2f}ms, {bt.call_count} appels")

## 10. Exercices

### Exercice 1 : Hidden Pair
Implementez la technique **Hidden Pair** : deux valeurs qui ne peuvent aller que dans les memes deux cases d'une unite.

### Exercice 2 : Swordfish
Implementez la technique **Swordfish** : extension du X-Wing avec 3 lignes/colonnes au lieu de 2.

### Exercice 3 : XY-Wing
Implementez la technique **XY-Wing** : utilise une case pivot avec 2 candidats et deux autres cases partageant chacune un candidat avec le pivot.

## Resume

### Techniques implementees

| Technique | Description | Difficulte |
|-----------|-------------|-----------|
| **Naked Single** | Case avec un seul candidat | Facile |
| **Hidden Single** | Valeur avec une seule position possible | Facile |
| **Naked Pair** | Deux cases avec les memes deux candidats | Moyen |
| **Pointing Pair** | Candidats restreints dans un bloc | Moyen |
| **X-Wing** | Rectangle de candidats | Avance |

### Strategies par difficulte

| Niveau | Techniques requises |
|--------|-------------------|
| Easy | Naked Singles, Hidden Singles |
| Medium | + Naked Pairs, Pointing Pairs |
| Hard | + X-Wing |
| Expert | + Swordfish, XY-Wing, etc. |

### Avantages des strategies humaines

1. **Intuitives** : Refletent la facon dont les humains resolvent
2. **Pedagogiques** : Chaque technique est comprehensible individuellement
3. **Progressives** : Des techniques simples aux avancees
4. **Visuelles** : Peuvent etre expliquees avec des diagrammes

### Limitations

1. **Pas toujours completes** : Certains puzzles necessitent le backtracking
2. **Plus complexes** : Plus difficiles a implementer que le backtracking simple
3. **Performance** : Plus lentes que les solveurs CSP industriels

### References

- [SudokuWiki](https://www.sudokuwiki.org/) - Techniques de resolution
- [Hodoku](https://hodoku.sourceforge.net/) - Techniques et explications

---

**Navigation** : [<< Sudoku-6-AIMA-CSP](Sudoku-6-AIMA-CSP-Python.ipynb) | [Index](README.md) | [Sudoku-9-GraphColoring >>](Sudoku-9-GraphColoring-Python.ipynb)